In [ ]:
import json
import my
import pandas as pd
import sqlalchemy
import requests
from datetime import datetime, date, timedelta, timezone

def get_weather(cities_df):
    
    key = my.OW_API_key
    
    weather_dict = {'city_id':[],
                'time':[],
                'temp':[],
                'outlook':[],
                'wind_speed':[],
                }

    for i, city in enumerate(cities_df['city_id']):
        url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities_df.iloc[i]['city']}&appid={key}&units=metric"
        response = requests.get(url)
        weather_json = response.json()

        for time in weather_json['list']:
            weather_dict['city_id'].append(city)
            weather_dict['time'].append(time['dt_txt'])
            weather_dict['temp'].append(time['main']['temp'])
            weather_dict['outlook'].append(time['weather'][0]['main'])
            weather_dict['wind_speed'].append(time['wind']['speed'])
          
    
    weather_df = pd.DataFrame(weather_dict)
    return weather_df
    

def get_flights(airports_df):
    
    querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"true"}
    headers = {
    	"X-RapidAPI-Key": "Your key here",
    	"X-RapidAPI-Host": "Your host here"
    }
    
    




    # Create a timezone object for Europe/Berlin
    berlin_timezone = timezone(timedelta(hours=2))  # UTC+2 for Central European Summer Time (CEST)

    # Get the current time in the Berlin time zone
    current_time_berlin = datetime.now(berlin_timezone)

    # Get the date for today and tomorrow in Berlin
    today = current_time_berlin.date()
    tomorrow = today + timedelta(days=1)


    
    
    
    
    
    times = [["00:00","11:59"],["12:00","23:59"]]
    icao_list = airports_df['icao'].to_list()
    
    flights_dict = {'ICAO':[],
                'departure_airport':[],
                'local_time':[]}

    for time in times:
        
        for icao in icao_list[0:]:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            response = requests.request("GET", url, headers=headers, params=querystring)
            print(response)
            flight_json = response.json()
            
            for flight in flight_json['arrivals']:
                flights_dict['ICAO'].append(icao)
                try:
                    flights_dict['departure_airport'].append(flight['movement']['airport']['icao'])
                except:
                    flights_dict['departure_airport'].append('unknown')
                try:
                    flights_dict['local_time'].append(flight['movement']['scheduledTime']["local"])
                except:
                    flights_dict['local_time'].append(pd.NaT)
                    
    flights_df = pd.DataFrame(flights_dict)
    return flights_df

def lambda_handler(event, context):
    key = my.OW_API_key
    password = my.sql_key
    
    host = 'Your host here'
    schema = 'Your schema name here'
    user = 'Your user name here'
    port = "Your port number here"
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    cities_df = pd.read_sql('cities',con=con)
    weather_df = get_weather(cities_df)
    weather_df['time'] = pd.to_datetime(weather_df['time'])
    weather_df.to_sql('weather',con=con,if_exists='append',index=False)
    
    
    
    
    querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"true"}
    headers = {
    	"X-RapidAPI-Key": "Your Key Here",
    	"X-RapidAPI-Host": "Your host here"
    }
    
    airports_df = pd.read_sql('airports',con=con)
    flights_df = get_flights(airports_df)
    flights_df['local_time'] = pd.to_datetime(flights_df['local_time'])
    flights_df.to_sql('flights',con=con,if_exists='append',index=False)
    
    
    # TODO implement
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }
